# Streaming

Streaming...

There are two types frequently used with Agents:


In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

In [3]:
model = init_chat_model("gpt-5-mini", model_provider="openai")

agent = create_agent(
    model=model,
    tools=[],
    prompt=SystemMessage(content="You are a full-stack comedian"),
)

## No Steaming (invoke)

In [4]:
result = agent.invoke(
    {"messages": [{"role": "user", 
                   "content": "Tell me a joke"}]}
    )
print(result["messages"][1].content)

Why did the full-stack developer go broke?

Because he used up all his cache.

Want another one?


## value

In [5]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", 
                   "content": "Tell me a joke"}]},
    stream_mode="values",
    ):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Tell me a joke
================================== Ai Message ==================================

I’d tell you a UDP joke, but I’m not sure you’d get it.

Want another—TCP, HTTP, or something non-tech?


## message

In [6]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a joke."}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content}")

node: agent
content: 
node: agent
content: Why
node: agent
content:  did
node: agent
content:  the
node: agent
content:  web
node: agent
content:  developer
node: agent
content:  go
node: agent
content:  broke
node: agent
content: ?
node: agent
content:  Because
node: agent
content:  he
node: agent
content:  used
node: agent
content:  up
node: agent
content:  all
node: agent
content:  his
node: agent
content:  cache
node: agent
content: .
node: agent
content: 


In [7]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

The server hums a low applause, the browser blushes blue,
I stand between the markup and the midnight, telling jokes to you.
CSS fluffs up its ruffles; JavaScript trips on a grin —
I deploy a punchline, hit refresh, and watch the laughter spin.

My backend keeps the secrets, polite and tightly spun,
SQLs whisper algorithms, telling puns in rows of one.
The cache remembers yesterday’s best bits and then forgets,
So I git-commit a new riff — wild, imperfect, yet.

A merge conflict in the chorus, a race condition in the rhyme,
I catch exceptions with a wink and log the memory of time.
The stack trace is my set list, each line a tiny spark,
From localhost to starlight, from studio to dark.

Listeners are sockets open, awaiting something kind —
A bit of warm absurdity to loosen up the mind.
I debug subtle sorrows with a well-timed little gag,
And patch the holes with laughter, one soft semicolon tag.

So here’s my deploy: a poem, full-stack and slightly wrong,
Built with coffee, odd optimis

## Tools can stream too!

In [1]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]

):
    print(chunk)

('updates', {'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 132, 'total_tokens': 350, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CNhIWNs9mupBpZkKs77237j5elaaI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--5e224b37-a175-4908-bf88-cc625789e6a1-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco, CA'}, 'id': 'call_Doq6ytEN8PooMN0KJx4Ln8u7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 132, 'output_tokens': 218, 'total_tokens': 350, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [5]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]

):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco, CA
Acquired data for city: San Francisco, CA
